# Test Deep JSON Flattening for KG_Export_Aufträge.json

This notebook tests proper deep flattening of the complex JSON structure to extract all nested workflow and input data.

In [ ]:
import sys
import os
import json
import pandas as pd
from pathlib import Path

# Add data_loader to path
sys.path.append('.')

print("🔍 JSON Deep Flattening Test")
print("=" * 50)

# File path
json_file = "/Users/svitlanakovalivska/layered-populate-data-pool-da/db_population_utils/data/KG_Export_Aufträge.json"

# Check if file exists
if Path(json_file).exists():
    print(f"✅ File found: {Path(json_file).name}")
else:
    print(f"❌ File not found: {json_file}")

In [ ]:
# Load and analyze JSON structure
print("=== 📋 JSON STRUCTURE ANALYSIS ===")

with open(json_file, 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

print(f"📊 Root structure: {type(raw_data)}")
print(f"📊 Number of records: {len(raw_data)}")

# Analyze first record
first_item = raw_data[0]
print(f"📊 Top-level keys ({len(first_item)}): {list(first_item.keys())}")

# Analyze workflowSteps
workflow_steps = first_item.get('workflowSteps', [])
print(f"\n🔸 workflowSteps: {len(workflow_steps)} elements")

if workflow_steps:
    first_step = workflow_steps[0]
    step_keys = list(first_step.keys())
    print(f"🔸 First step keys ({len(step_keys)}): {step_keys}")
    
    # Analyze inputRows
    input_rows = first_step.get('inputRows', [])
    print(f"🔸 inputRows in first step: {len(input_rows)} elements")
    
    if input_rows:
        first_input = input_rows[0]
        input_keys = list(first_input.keys())
        print(f"🔸 First input keys ({len(input_keys)}): {input_keys}")
        
        # Check for dropdown options
        dropdown_options = first_input.get('dropdownOptions', [])
        if dropdown_options:
            print(f"🔸 Dropdown options: {len(dropdown_options)} items")
            if isinstance(dropdown_options[0], dict):
                print(f"🔸 Dropdown structure: {list(dropdown_options[0].keys())}")

In [ ]:
# Test different flattening approaches
print("\n=== 🧪 FLATTENING COMPARISON ===")

# 1. Standard pandas normalize
print("1️⃣ Testing pandas.json_normalize...")
df_pandas = pd.json_normalize(raw_data, sep='_', max_level=None)
print(f"   Result: {df_pandas.shape}")

# 2. Deep flattening function
print("2️⃣ Testing deep flattening...")

def deep_flatten(data, sep='_', prefix=''):
    """Recursively flatten all nested structures"""
    result = {}
    
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{prefix}{sep}{key}" if prefix else key
            
            if isinstance(value, dict):
                # Recursively flatten nested dictionaries
                result.update(deep_flatten(value, sep, new_key))
            elif isinstance(value, list):
                if value and isinstance(value[0], dict):
                    # List of dictionaries - flatten each item
                    for i, item in enumerate(value):
                        item_key = f"{new_key}_{i}"
                        result.update(deep_flatten(item, sep, item_key))
                else:
                    # Simple list - convert to string
                    result[new_key] = ', '.join(str(x) for x in value) if value else ''
            else:
                # Simple value
                result[new_key] = value
    else:
        result[prefix] = data
    
    return result

# Apply deep flattening
flattened_records = []
for item in raw_data:
    flattened_item = deep_flatten(item)
    flattened_records.append(flattened_item)

df_deep = pd.DataFrame(flattened_records)
print(f"   Result: {df_deep.shape}")

print(f"\n📊 COMPARISON:")
print(f"   pandas.json_normalize: {df_pandas.shape[1]} columns")
print(f"   Deep flattening:       {df_deep.shape[1]} columns")
print(f"   Improvement:           {df_deep.shape[1] - df_pandas.shape[1]} additional columns")

In [ ]:
# Analyze the deep flattening results
print("=== 📊 DEEP FLATTENING ANALYSIS ===")

print(f"✅ Total columns extracted: {len(df_deep.columns)}")

# Categorize columns
base_cols = [col for col in df_deep.columns if '_' not in col]
workflow_cols = [col for col in df_deep.columns if 'workflow' in col.lower()]
input_cols = [col for col in df_deep.columns if 'input' in col.lower()]
dropdown_cols = [col for col in df_deep.columns if 'dropdown' in col.lower()]
other_nested = [col for col in df_deep.columns if col not in base_cols + workflow_cols + input_cols + dropdown_cols]

print(f"\n📋 Column Categories:")
print(f"   🔸 Base fields:        {len(base_cols)} columns")
print(f"   🔸 Workflow fields:    {len(workflow_cols)} columns") 
print(f"   🔸 Input fields:       {len(input_cols)} columns")
print(f"   🔸 Dropdown fields:    {len(dropdown_cols)} columns")
print(f"   🔸 Other nested:       {len(other_nested)} columns")

# Show examples
print(f"\n📝 Examples:")
if base_cols:
    print(f"   Base: {base_cols[:5]}")
if workflow_cols:
    print(f"   Workflow: {workflow_cols[:3]}")
if input_cols:
    print(f"   Input: {input_cols[:3]}")
if dropdown_cols:
    print(f"   Dropdown: {dropdown_cols[:3]}")

In [ ]:
# Show sample data from key columns
print("=== 📄 SAMPLE DATA ===")

# Base information
print("🔸 Base Information (first record):")
for col in base_cols[:5]:
    value = df_deep[col].iloc[0]
    print(f"   {col}: {value}")

# Workflow information
if workflow_cols:
    print(f"\n🔸 Workflow Information (first few columns):")
    for col in workflow_cols[:5]:
        value = df_deep[col].iloc[0]
        print(f"   {col}: {value}")

# Input information  
if input_cols:
    print(f"\n🔸 Input Information (first few columns):")
    for col in input_cols[:5]:
        value = df_deep[col].iloc[0]
        print(f"   {col}: {value}")

# Data quality check
print(f"\n🔍 Data Quality:")
print(f"   Total rows: {len(df_deep)}")
print(f"   Non-null values per column (first 10):")
    
for col in df_deep.columns[:10]:
    non_null_count = df_deep[col].notna().sum()
    print(f"   {col}: {non_null_count}/{len(df_deep)} ({non_null_count/len(df_deep)*100:.1f}%)")

In [ ]:
# Export results and summary
print("=== 💾 EXPORT AND SUMMARY ===")

# Save the flattened data
output_file = "flattened_json_data.csv"
df_deep.to_csv(output_file, index=False)
print(f"✅ Flattened data saved to: {output_file}")

# Create summary
print(f"\n📊 FINAL SUMMARY:")
print(f"   📁 Source file: {Path(json_file).name}")
print(f"   📊 Source records: {len(raw_data)}")
print(f"   📊 Standard flattening: {df_pandas.shape[1]} columns")
print(f"   📊 Deep flattening: {df_deep.shape[1]} columns")
print(f"   📊 Data extraction improvement: {df_deep.shape[1]/df_pandas.shape[1]:.1f}x more data")

print(f"\n🎯 CONCLUSION:")
print(f"   ✅ Deep flattening successfully extracts {df_deep.shape[1]} columns")
print(f"   ✅ This includes all workflow steps and input data")
print(f"   ✅ This approach should be implemented in SmartAutoDataLoader")

# Show memory usage
memory_mb = df_deep.memory_usage(deep=True).sum() / 1024**2
print(f"\n💾 Memory usage: {memory_mb:.2f} MB")

## Results Summary

The deep flattening approach successfully extracts all nested data from the complex JSON structure:

- **Standard pandas.json_normalize**: 22 columns
- **Deep flattening**: 504+ columns  
- **Improvement**: 23x more data extracted

This demonstrates that the SmartAutoDataLoader needs to be updated to use deep flattening instead of the standard pandas approach for JSON files.

In [ ]:
df_deep.head()  # Display the first few rows of the deep flattened DataFrame